# Part One: handle multiplication on CPU

Let's implement an autograd library that can handle multiplication:

a = Tensor(np.array([2]))
b = Tensor(np.array([3]))
c = a.mul(b)
c.backward()

We will expect a.grad to be [3] and b.grad = to be [2]

We use numpy as the underlying library to handle the actual operations, what we build here is how to calculate gradient automatically. Let's first import it.

In [ ]:
import numpy as np

We start off with a Tensor class that takes numpy array as input, it will have the mul and backward method:

In [ ]:
class Tensor:
  def __init__(self, data):
    self.data = data

  def __repr__(self):
    return f"<Tensor with data: {self.data}, grad: {self.grad}>"
  
  def mul(self, other):
    pass
  
  def backward(self):
    pass

The multiplication method will be handled by just calling the numpy operations:

In [ ]:
def mul(self, other):
    out_data = self.data * other.data # np.array([2]) * np.array([3])
    out_tensor = Tensor(out_data)
    return out_tensor

Having just the calculation is not enough, we need to keep track of the operations so we can do backpropagation. The idea is to pass in the the two tensors that formed the new tensor as an argument, and then do a recursive search for the calculation graph. 

In [8]:
class Tensor:
  def __init__(self, data, children=()) -> None:
    self.data = data
    self._prev = set(children)

  def mul(self, other):
    out = Tensor(self.data * other.data, (self, other))
    return out
  
  def build_topo(self):
    topo = []
    visited = set()
    def _build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          _build_topo(child)
        topo.append(v)
    _build_topo(self)
    return topo

  def __repr__(self):
    return f"<Tensor with data: {self.data}>"
  

With this new tensor, we can actually show the calculation graph. Note that I added a __repr__ method to show the content of this class

In [9]:
a = Tensor(np.array([2]))
b = Tensor(np.array([3]))
c = a.mul(b)
tree = c.build_topo()
print(tree)

[<Tensor with data: [3]>, <Tensor with data: [2]>, <Tensor with data: [6]>]


We can try with more complicated examples

In [11]:

a = Tensor(np.array([2]))
b = Tensor(np.array([3]))
c = a.mul(b)
d = c.mul(b)
e = d.mul(c)
tree = e.build_topo()
print(tree)

[<Tensor with data: [3]>, <Tensor with data: [2]>, <Tensor with data: [6]>, <Tensor with data: [18]>, <Tensor with data: [108]>]


What we have now is a mechanism to record all the tensors that contributed to the final value, however, the relationship between how those tensors are operated are not kept. Next, we will implement the actual backpropagation to fill in that missing piece.

First, we add two more attributes to the tensor class. One is grad, this records the current gradient of this tensor, each tensor will have a _backward attributes as a function that perform updates to its own gradient. This function is only defined when operations like multiplication is performed on it, hence capturing the relationship between the two tensors that were combined to form it.

In [ ]:
class Tensor:
  def __init__(self, data, children=()) -> None:
    self.data = data
    self._prev = set(children)
    self.grad = np.zeros(self.data.shape)
    self._backward = lambda: None

Next let's see how the `mul` method can capture the 3 * 2 operations and update the gradient accordingly:

In [13]:
class Tensor:
  def __init__(self, data, children=()) -> None:
    self.data = data
    self._prev = set(children)
    self.grad = np.zeros(self.data.shape)
    self._backward = lambda: None

  def mul(self, other):
    out = Tensor(self.data * other.data, (self, other))
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out
  
  def __repr__(self):
    return f"<Tensor with data: {self.data}, grad: {self.grad}>"

Let's play around with this to understand what it does. In the following example, c = a * b, meaning that the gradient of a is the .data on b, and the gradient of b is the .data on a. If we want to have the value show up on the a and b instance, we can call c._backward()

In [14]:
a = Tensor(np.array([2]))
b = Tensor(np.array([3]))
c = a.mul(b)
c._backward()
print(a)
print(b)

<Tensor with data: [2], grad: [0.]>
<Tensor with data: [3], grad: [0.]>


Oops, it's not what we expect, reason is that the initial gradient of c is zero, so when you multiply it by other values, it is still zero. However, the initial gradient of a and b needs to be 0. To fix it, we initialize the gradient of our "starting" tensor with 1:

In [15]:
a = Tensor(np.array([2]))
b = Tensor(np.array([3]))
c = a.mul(b)
c.grad = np.ones(c.data.shape)
c._backward()
print(a)
print(b)

<Tensor with data: [2], grad: [3.]>
<Tensor with data: [3], grad: [2.]>


Here we have the expected result. In fact, we have already implemented the backpropgation, we just didn't wrap it into a proper method and have it applied on every tensor, let's do that:

In [16]:
class Tensor:
  def __init__(self, data, children=()) -> None:
    self.data = data
    self._prev = set(children)
    self.grad = np.zeros(self.data.shape)
    self._backward = lambda: None

  def mul(self, other):
    out = Tensor(self.data * other.data, (self, other))
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out
    
  def build_topo(self):
    topo = []
    visited = set()
    def _build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          _build_topo(child)
        topo.append(v)
    _build_topo(self)
    return topo

  def backward(self):
    self.grad = np.ones(self.data.shape)
    tree = self.build_topo()
    for tensor in reversed(tree):
      tensor._backward()
  
  def __repr__(self):
    return f"<Tensor with data: {self.data}, grad: {self.grad}>"
  

<Tensor with data: [2 3], grad: [ 9. 16.]>
<Tensor with data: [3 4], grad: [12. 24.]>


Let's see the result now, as you can see, it handles multi element tensors and multiple operations well:

In [ ]:
a = Tensor(np.array([2,3]))
b = Tensor(np.array([3,4]))
c = a.mul(b)
d = c.mul(b)
d.backward()
print(a)
print(b)

This is an autograd library in its simpliest form. Recall in the title I said we are making this run on CPU? that's actually numpy. When we do `self.data * other.grad`, the __mul__ method is provided by the numpy library, which runs on CPU in the form of JIT compiled C++ code. Because the __mul__ happens on numpy, we don't have to worry it about it polluting our _backward function that's updated with every call to `.mul()`. In order for this to work on GPU, and allows for easy transitioning between the two device type, we will need to have something like numpy, but running on GPU. There are different ways to do this, and in the next notebook, I will present the approach tinygrad has taken.